In [1]:
import itertools
import functools
import operator
import re
import xarray
import numpy as np
from dataclasses import dataclass

# Day 1

### Part 1

In [61]:
with open('Advent_2020_Day_1.txt') as f:
    nums = f.read().splitlines()
nums = [int(x) for x in nums]

In [62]:
@dataclass
class Coord:
    a: int
    b: int

In [63]:
pt = Coord(0,1)

while pt.b <= len(nums)-1 and pt.a <= len(nums)-1:
    f,s = int(nums[pt.a]), int(nums[pt.b])
    if f+s == 2020:
        print(pt.a,pt.b)
        print("Sum:",f+s)
        print("Product:", f*s)
        break
        
    #Check if a and b are the same, else add 1
    if pt.a == pt.b:
        pt.b += 1
        # print('1 added to b')
        continue

    #Check if b is at the end, else add 1 to a and reset b
    if pt.b == len(nums) -1:
        pt.a+=1
        pt.b = pt.a + 1
        # print('1 added to a')
        # print('b reset')
        continue
    #Else, add one to b
    pt.b += 1
    # print('1 added to b')
    continue

34 159
Sum: 2020
Product: 1018944


### Part 2: Itertools for conv

In [69]:
comb = list(itertools.combinations(nums,3))
res = [i for i in comb if sum(i) == 2020][0]

In [74]:
functools.reduce(operator.mul,res)

8446464

# Day 2

### Part 1

In [76]:
with open("./Advent_2020_Day_2.txt") as f:
    data = f.read().splitlines()

In [135]:
class instruct:
    def __init__(self,instruction: str):
        
        pattern = "(\d+)-(\d+) (\w): (\w+$)"
        m = re.match(pattern,instruction)
        
        self.min = int(m.group(1))
        self.max = int(m.group(2))
        self.char = m.group(3)
        self.pw = m.group(4)
        
    def validate_pw(self):
        count = self.pw.count(self.char)
        return (count>=self.min and count<=self.max)
    
    def validate_pw_new(self):
        pair = str(self.pw[self.min-1]) + str(self.pw[self.max-1])
        return pair.count(self.char) == 1

In [133]:
valid = sum([instruct(i).validate_pw() for i in data])
valid

524

### Part 2

In [137]:
valid = sum([instruct(i).validate_pw_new() for i in data])
valid

485

# Day 3

### Part 1

In [98]:
with open("./Advent_2020_Day_3.txt") as f:
    data = f.read().splitlines()

In [105]:
#Parse map into array with coords
dmap = list(map(list,data))
dmap = xarray.DataArray(dmap, dims = ["row","col"])

In [179]:
class Explorer:
    def __init__(self,row: int,col: int, dmap: xarray.DataArray):
        self.row = row
        self.col = col
        self.dmap = dmap
        self.treecount = 0
        
    def move_single(self,direction: str, distance: int):
        if direction == "r":
            self.col += 1
        elif direction == "l":
            self.col -= 1
        elif direction == "u":
            self.row += -1
        elif direction == "d":
            self.row += 1
        else:
            raise Exception("Direction unknown")
            
    def validate_position_in_map(self):
        """Checks if current position is still in map
        """
        rowcheck = self.row in dmap.get_index("row")
        colcheck = self.col in dmap.get_index("col")
        
        if rowcheck and colcheck:
            return True
        else:
            return False
    
    def fix_position(self):
        """Fixes position relative to map, since the map can be stacked right-ward.
        """
        if self.col >= dmap.get_index("col").max():
            self.col -= (dmap.get_index("col").max() + 1)
        elif self.row not in dmap.get_index("row"):
            pass
            # raise Exception("Can't be fixed: dropped off row axis")
            
        
    def move_multiple(self,pattern):
        """Moves according to specified pattern, and records if tree is at the end
        """
        
        #Check if still in map, and fix if needed
        if self.validate_position_in_map() == False:
            self.fix_position()       
            
        for i in pattern:
            #Move           
            self.move_single(i,1)
        
        #Check again if still in map, and fix if needed
        if self.validate_position_in_map() == False:
            self.fix_position()  
            
        #Look for tree
        self.record_tree()
        
    def record_tree(self):
        if self.validate_position_in_map() == True:
            if dmap[self.row,self.col] == "#":
                self.treecount += 1

In [180]:
dude = Explorer(0,0,dmap)

while dude.row <= dmap.get_index("row").max():
    dude.move_multiple("rrrd")
dude.treecount

234

### Part 2

In [181]:
digs = []
for pattern in ["rd","rrrd","rrrrrd","rrrrrrrd","rdd"]:
    dude = Explorer(0,0,dmap)
    while dude.row <= dmap.get_index("row").max():
        dude.move_multiple(pattern)
    digs.append(dude.treecount)

In [183]:
functools.reduce(operator.mul, digs)

5813773056

# Day 4

### Part 1

In [23]:
with open("./Advent_2020_Day_4.txt") as f:
    data = f.read().splitlines()

In [29]:
#Parse data
full_data = []
tracker = []
for e in data:
    if e == "":
        full_data.append(tracker)
        tracker = []
        pass
    else:
        tracker.append(e)

#Add last entry
full_data.append(tracker)

In [83]:
teststr = full_data[0][0]
pattern = r"((?P<key>\w+):(?P<value>(?:\w|\d)+)\s?)+"

match = re.match(pattern,teststr)

In [84]:
match

<re.Match object; span=(0, 34), match='eyr:2028 iyr:2016 byr:1995 ecl:oth'>

In [93]:
match.group("key")

'ecl'

In [32]:
def str_to_dict(string:str) -> dict:
    """Converts passport input to a dict
    """
    pattern = r"((\w+):((\w|\d)+)\s?)+"

In [31]:
full_data

[['eyr:2028 iyr:2016 byr:1995 ecl:oth',
  'pid:543685203 hcl:#c0946f',
  'hgt:152cm',
  'cid:252'],
 ['hcl:#733820 hgt:155cm',
  'iyr:2013 byr:1989 pid:728471979',
  'ecl:grn eyr:2022'],
 ['hgt:171cm', 'iyr:2013 pid:214368857 hcl:#cfa07d byr:1986 eyr:2028 ecl:grn'],
 ['hgt:167cm cid:210 ecl:brn pid:429131951 hcl:#cfa07d eyr:2029 iyr:2010',
  'byr:1945'],
 ['hcl:#888785 iyr:2015', 'hgt:170cm pid:893805464 ecl:amb byr:1966 eyr:2028'],
 ['hgt:170cm ecl:amb',
  'hcl:#c0946f eyr:2020 iyr:2016 pid:725010548',
  'byr:1928'],
 ['byr:1999 hcl:#888785',
  'eyr:2026',
  'ecl:hzl',
  'iyr:2016 hgt:193cm pid:170608679'],
 ['eyr:2024 iyr:2016 hcl:#cfa07d ecl:grn byr:2001 pid:391942873 cid:104 hgt:164cm'],
 ['iyr:2019',
  'eyr:2025 pid:138912840 byr:1996',
  'hgt:166cm',
  'hcl:#888785 ecl:grn'],
 ['iyr:2023 hcl:a58381 pid:#401a29 eyr:1940', 'byr:1920', 'ecl:utc hgt:183cm'],
 ['pid:493510244 ecl:gry hgt:153cm byr:1950 cid:181 eyr:2028',
  'hcl:#ceb3a1',
  'iyr:2020'],
 ['iyr:2018 pid:074340974 hgt:18